In [34]:
import numpy as np
import pandas as pd

In [35]:
training_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')

training_data.drop(columns='Id',inplace=True)
test_data.drop(columns='Id',inplace=True)

y_train = training_data['SalePrice']
training_data.drop(columns='SalePrice',inplace=True)

N_train = training_data.shape[0]
N_test = training_data.shape[0]

# Join train and test data for preprocessing.
dataset = pd.concat((training_data,test_data),axis=0)

In [36]:
# Find and deal with NaN values by consulting the dataset documentation.
# First fill nan with None where it indicates the given feature is not present

feats = ['Alley',
         'BsmtQual',
         'BsmtCond',
         'BsmtExposure',
         'BsmtFinType1',
         'BsmtFinType2',
         'FireplaceQu',
         'GarageType',
         'GarageFinish',
         'GarageQual',
         'GarageCond',
         'PoolQC',
         'Fence',
         'MiscFeature']

for f in feats:
    dataset[f].fillna('None',inplace=True)


# Enforce zero for certain ordinal features relating to a part of the structure which may not exist
for f in ['BsmtFullBath','BsmtHalfBath','BsmtFinSF1','BsmtFinSF2','BsmtUnfSF','TotalBsmtSF']:
    dataset.loc[dataset['BsmtQual']=='None',f] = 0


for f in ['GarageCars','GarageArea','GarageFinish']:
    dataset.loc[dataset['GarageCond']=='None',f] = 0

 # GarageYrBlt is a potentially problematic feature which we will revisit. Leave as NaN for now.

print('Remaining features containing NaN:')
print(list(dataset.columns[dataset.isna().sum()>0]))



Remaining features containing NaN:
['MSZoning', 'LotFrontage', 'Utilities', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'MasVnrArea', 'Electrical', 'KitchenQual', 'Functional', 'GarageYrBlt', 'SaleType']


We now investigate the remaining nan-containing features. We can check if NaN again indicates the lack of a given house feature (for example, Masonry Veneer) by checking if all features concerning that house feature are also NaN. Then we can fill nan's appropriately.

In [37]:
# MasVnrType contains lots of NaN entries, which also have corresponding NaNs for MasVnrArea.
# Thus we conclude that in this case, NaN corresponds to no masonry veneer
print(dataset[dataset['MasVnrArea'].isna()][['MasVnrArea','MasVnrType']].head(100))
dataset['MasVnrArea'].fillna(0,inplace=True)
dataset['MasVnrType'].fillna('None',inplace=True)

      MasVnrArea MasVnrType
234          NaN        NaN
529          NaN        NaN
650          NaN        NaN
936          NaN        NaN
973          NaN        NaN
977          NaN        NaN
1243         NaN        NaN
1278         NaN        NaN
231          NaN        NaN
246          NaN        NaN
422          NaN        NaN
532          NaN        NaN
544          NaN        NaN
581          NaN        NaN
851          NaN        NaN
865          NaN        NaN
880          NaN        NaN
889          NaN        NaN
908          NaN        NaN
1132         NaN        NaN
1197         NaN        NaN
1226         NaN        NaN
1402         NaN        NaN


In [38]:
# LotFrontage contains NaNs, but no zeros. Likely that properties with 0 LotFrontage have been recorded as NaN
print()
print(f'{dataset[dataset["LotFrontage"]==0].shape[0]} entries with LotFrontage == 0')

# Replace Nan with 0 for LotFrontage
dataset['LotFrontage'].fillna(0,inplace=True)


0 entries with LotFrontage == 0


Investigate NaN in KitchenQual. There is only one NaN entry, which does not correspond to a property without a kitchen. We replace it with the mode value of KitchenQual

In [39]:
dataset[dataset['KitchenAbvGr']==0][['KitchenAbvGr', 'KitchenQual']]

,KitchenAbvGr,KitchenQual
954,0,TA
1127,0,TA
1399,0,TA


In [40]:
dataset[dataset['KitchenQual'].isna()][['KitchenAbvGr', 'KitchenQual']]

,KitchenAbvGr,KitchenQual
95,1,NaN


In [41]:
dataset['KitchenQual'].fillna(dataset['KitchenQual'].mode()[0],inplace=True)

# For Functional feature, documentation states to assume Typ. We replace NaN with 'Typ'
dataset['Functional'].fillna('Typ',inplace=True)

print('Remaining features containing NaN:')
print(list(dataset.columns[dataset.isna().sum()>0]))

Remaining features containing NaN:
['MSZoning', 'Utilities', 'Exterior1st', 'Exterior2nd', 'Electrical', 'GarageYrBlt', 'SaleType']


In [42]:
# Perform mode imputation for remaining NaN-containing features (other than GarageYrBlt).

for f in list(dataset.columns[dataset.isna().sum()>0]):
    if f != 'GarageYrBlt':
        dataset[f].fillna(dataset[f].mode()[0],inplace=True)
        

# MSSubClass is a numerical feature which should not be interpreted as an ordinal data type. We cast it to string:
dataset['MSSubClass'] = dataset['MSSubClass'].astype('str')


# Use sine/cosine representation of month sold and remove original feature
dataset['cos(MoSold)'] = np.cos(dataset['MoSold']*2*np.pi/12)
dataset['sin(MoSold)'] = np.sin(dataset['MoSold']*2*np.pi/12)
dataset.drop(columns='MoSold',inplace=True)

In [43]:
# Convert string-denoted features to interger-denoted ordinals where appropriate
# (ie the feature represents increasing 'quality')

ordinals = ['ExterQual',
            'ExterCond',
            'BsmtQual',
            'BsmtCond',
            'KitchenQual',
            'HeatingQC',
            'FireplaceQu',
            'GarageQual',
            'GarageCond',
            'PoolQC']

# BsmtExposure
for fname in ordinals:
    dataset[fname].replace(['Ex', 'Gd', 'TA', 'Fa', 'Po', 'None'],
                           [5,4,3,2,1,0], inplace=True)
    
    dataset[fname].replace(['Gd', 'Av', 'Mn', 'No'],
                           [3,2,1,0], inplace=True)

dataset['Street'].replace(['Grvl','Pave'],[0,1], inplace=True)

dataset['Alley'].replace(['None', 'Grvl','Pave'],[0,1,2], inplace=True)

dataset['LandSlope'].replace(['Gtl','Mod', 'Sev'],
                             [2,1,0], inplace=True)

dataset['BsmtExposure'].replace(['Gd','Av','Mn','No','None'],
                         [4,3,2,1,0], inplace=True)

dataset['BsmtFinType1'].replace(['GLQ','ALQ', 'BLQ', 'Rec', 'LwQ', 'Unf', 'None'],
                                [6,5,4,3,2,1,0], inplace=True)

dataset['BsmtFinType2'].replace(['GLQ','ALQ', 'BLQ', 'Rec', 'LwQ', 'Unf', 'None'],
                                [6,5,4,3,2,1,0], inplace=True)

dataset['CentralAir'].replace(['Y','N'],[1,0], inplace=True)

dataset['Functional'].replace(['Typ','Min1', 'Min2', 'Mod', 'Maj1', 'Maj2', 'Sev', 'Sal'],
                              [7,6,5,4,3,2,1,0], inplace=True)

dataset['GarageFinish'].replace(['Fin','RFn', 'Unf', 'None'],
                                [3,2,1,0], inplace=True)

dataset['PavedDrive'].replace(['Y','P','N'],
                              [2,1,0], inplace=True)

dataset['Fence'].replace(['GdPrv','MnPrv', 'GdWo', 'MnWw', 'None'],
                         [4,3,2,1,0], inplace=True)


ordinals = ordinals+ ['Street',
                      'Alley',
                      'LandSlope',
                      'BsmtExposure',
                      'BsmtFinType1',
                      'BsmtFinType2',
                      'CentralAir',
                      'Functional',
                      'GarageFinish',
                      'PavedDrive',
                      'Fence']

for fname in ordinals:
    print(fname)
    dataset[fname].astype(dtype=int,copy=False)


ExterQual
ExterCond
BsmtQual
BsmtCond
KitchenQual
HeatingQC
FireplaceQu
GarageQual
GarageCond
PoolQC
Street
Alley
LandSlope
BsmtExposure
BsmtFinType1
BsmtFinType2
CentralAir
Functional
GarageFinish
PavedDrive
Fence


In [44]:
# Deal with GarageYrBlt by creating a new feature which indicates whether the garage was added after construction
dataset['NewGarage'] = (dataset['YearBuilt']!=dataset['GarageYrBlt']).astype('int')
dataset.drop(columns='GarageYrBlt',inplace=True)
# More feature creation
dataset['HasRemodel']      = (dataset['YearBuilt']!=dataset['YearRemodAdd']).astype('int')
dataset['HasGarage']       = (dataset['GarageType']!='None').astype('int')
dataset['HasPool']         = (dataset['PoolQC']>0).astype('int')
dataset['HasFence']         = (dataset['Fence']>0).astype('int')
dataset['HasFirePlace']    = (dataset['FireplaceQu']>0).astype('int')
dataset['HasMisc']    = (dataset['MiscFeature']!='None').astype('int')

dataset['HiQualFinSF']     = dataset['GrLivArea']-dataset['LowQualFinSF']
dataset['TotBathAG']       = dataset['FullBath'] + dataset['HalfBath']
dataset['TotBathBmt']      = dataset['BsmtFullBath'] + dataset['BsmtHalfBath']
dataset['TotBath']         = dataset['TotBathAG'] + dataset['TotBathBmt']
dataset['BsmtUnfFraction'] = dataset['BsmtUnfSF']/dataset['TotalBsmtSF']
dataset['RemodelDelta']    = dataset['YearRemodAdd']-dataset['YearBuilt']
dataset['AgeSold']         = dataset['YrSold']-dataset['YearBuilt']

dataset['BsmtUnfFraction'].fillna(value=0, inplace=True)




In [45]:
# Convert categorical features to a set of one-hot encoded features, inspect feature list
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
def categorical_to_ohe(input_data,feature_names):
    input_data = input_data.copy()
    le = LabelEncoder()
    ohe = OneHotEncoder()
    for fname in feature_names:
        tmp = ohe.fit_transform(le.fit_transform(input_data[fname]).reshape(-1,1)
                     ).toarray()

        input_data.drop(columns=fname,inplace=True)
        new_fnames = [f'{fname} = {c}' for c in le.classes_]
        input_data[new_fnames] = tmp
    return input_data



categorical_features = list(dataset.dtypes[dataset.dtypes=='object'].index)

dataset_processed = categorical_to_ohe(dataset, categorical_features)

list(dataset_processed.columns)

['LotFrontage',
 'LotArea',
 'Street',
 'Alley',
 'LandSlope',
 'OverallQual',
 'OverallCond',
 'YearBuilt',
 'YearRemodAdd',
 'MasVnrArea',
 'ExterQual',
 'ExterCond',
 'BsmtQual',
 'BsmtCond',
 'BsmtExposure',
 'BsmtFinType1',
 'BsmtFinSF1',
 'BsmtFinType2',
 'BsmtFinSF2',
 'BsmtUnfSF',
 'TotalBsmtSF',
 'HeatingQC',
 'CentralAir',
 '1stFlrSF',
 '2ndFlrSF',
 'LowQualFinSF',
 'GrLivArea',
 'BsmtFullBath',
 'BsmtHalfBath',
 'FullBath',
 'HalfBath',
 'BedroomAbvGr',
 'KitchenAbvGr',
 'KitchenQual',
 'TotRmsAbvGrd',
 'Functional',
 'Fireplaces',
 'FireplaceQu',
 'GarageFinish',
 'GarageCars',
 'GarageArea',
 'GarageQual',
 'GarageCond',
 'PavedDrive',
 'WoodDeckSF',
 'OpenPorchSF',
 'EnclosedPorch',
 '3SsnPorch',
 'ScreenPorch',
 'PoolArea',
 'PoolQC',
 'Fence',
 'MiscVal',
 'YrSold',
 'cos(MoSold)',
 'sin(MoSold)',
 'NewGarage',
 'HasRemodel',
 'HasGarage',
 'HasPool',
 'HasFence',
 'HasFirePlace',
 'HasMisc',
 'HiQualFinSF',
 'TotBathAG',
 'TotBathBmt',
 'TotBath',
 'BsmtUnfFraction',
 

In [46]:
# Save processed dataset
dataset_processed.iloc[:N_train,:].to_csv('./X_train.csv')
y_train.to_csv('./y_train.csv')
dataset_processed.iloc[N_train:,:].to_csv('./X_test.csv')